<a href="https://colab.research.google.com/github/jonathanhicks97/MLTSA22_JHicks/blob/main/HW8/HW8_JHicks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import pylab as pl

%pylab inline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel, ExpSineSquared
from scipy import interpolate
!pip install george
import george
from george import kernels


Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ls drive/MyDrive/MLTSA22/


 HW2/
 my_apis.csv
 New+york,ny.csv
'SP.POP.TOTL?downloadformat=excel'
'SP.POP.TOTL?downloadformat=excel.1'
'SP.POP.TOTL?downloadformat=excel.2'
'SP.POP.TOTL?downloadformat=excel.3'
'SP.POP.TOTL?downloadformat=excel.4'
'SP.POP.TOTL?downloadformat=excel.5'
 training_set.csv
 training_set_metadata.csv
 United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv


In [6]:
lcvs = pd.read_csv('drive/MyDrive/MLTSA22/training_set.csv')
lcvs.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [7]:
print("The shape of light curves dataset is:", lcvs.shape)

The shape of light curves dataset is: (1421705, 6)


In [8]:
bands = lcvs["passband"].unique()
print("Passbands (filter) identifiers:", bands)
objects = lcvs["object_id"].unique()
print("There are {} astrophysical simulated objects in my dataset".format(objects.shape[0]))

Passbands (filter) identifiers: [2 1 3 4 5 0]
There are 7848 astrophysical simulated objects in my dataset


In [50]:

if 'normalized' in globals():
    del normalized
reset_time = lcvs.groupby(['object_id','passband'])['mjd'].apply(lambda x: x - x.min())
lcvs['delta_t'] = reset_time.values




In [113]:
normalized = lcvs.groupby(['object_id','passband'], as_index = False)[['object_id','passband','delta_t','flux']]

In [146]:
TotalDF = pd.DataFrame(normalized)

In [178]:
TotalDF[1][1]

,object_id,mjd,passband,flux,flux_err,detected,delta_t
1,615,59750.4306,1,-816.434326,5.553370,1,0.0000
5,615,59752.4147,1,-1061.457031,6.472994,1,1.9841
10,615,59767.3045,1,-815.188599,5.293019,1,16.8739
15,615,59770.2256,1,-820.042786,5.875329,1,19.7950
20,615,59779.3265,1,-921.002502,6.306800,1,28.8959
25,615,59782.1974,1,-449.095612,4.028310,1,31.7668
30,615,59797.2938,1,35.511822,3.163646,1,46.8632
35,615,59800.3244,1,129.541901,4.358776,1,49.8938
40,615,59807.1815,1,-420.796417,4.037735,1,56.7509
45,615,59810.1122,1,-527.020325,4.204173,1,59.6816


In [239]:
rows = TotalDF.shape[0]

interpolated_func = [None] * rows
ynew = [None] * rows
object_id = [None] * rows
passband = [None] * rows
for i in range(rows):
  dataset = TotalDF[1][i]
  delta_t_new = np.linspace(0, dataset['delta_t'].max(), num = 50)
  interpolated_func[i] = interpolate.interp1d(x = dataset['delta_t'], y = dataset['flux'], kind = 'linear')
  ynew[i] = interpolated_func[i](delta_t_new)
  object_id[i] = dataset['object_id']
  passband[i] = dataset['passband']

In [234]:
for i in range(len(object_id)):
  object_id[i] = object_id[i].values[1]
  passband[i] = passband[i].values[1]

In [244]:
Final_Dataset = pd.DataFrame(data = [[delta_t_new, ynew, object_id, passband]], columns = ['t','flux','object','passband'])